# For-Norm Model 

For-norm is the second version of the Fake-or-Real dataset.
It contains the same speech files as the original version, but with important improvements:

Balanced in terms of:

* Gender (equal male/female voices)

* Class (equal real/fake examples)

Normalized in terms of:

* Sample rate (likely all files are 16kHz)

* Volume (loudness standardized)

* Number of channels (mono/stereo consistency)

Removes irrelevant differences between files (e.g., loud vs quiet).

Removes biases (e.g., too many fake male voices).

Helps your model learn real vs fake speech — not learn about bad recording conditions.

Training becomes faster, more stable, and gives higher accuracy.

In [1]:
#installing + importing 
!pip install transformers torchaudio tqdm

import os
import torch
import torchaudio
from transformers import Wav2Vec2Model, Wav2Vec2Processor
from tqdm import tqdm
import time


  Using cached torchaudio-2.6.0-cp39-cp39-manylinux1_x86_64.whl.metadata (6.6 kB)
  Using cached torch-2.6.0-cp39-cp39-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata 

/stor/home/spl742/wavenv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# defining paths
DATA_DIR = "deepfake_audio_project/data/raw/for-norm/for-norm"
SAVE_DIR = "deepfake_audio_project/data/processed_embeddings_norm"
os.makedirs(SAVE_DIR, exist_ok=True)

# checking device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [3]:
# load Wav2Vec2 model
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h").to(device)
model.eval()


Using the `SDPA` attention implementation on multi-gpu setup with ROCM may lead to performance issues due to the FA backend. Disabling it to use alternative backends.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Wav2Vec2Model(
  (feature_extractor): Wav2Vec2FeatureEncoder(
    (conv_layers): ModuleList(
      (0): Wav2Vec2GroupNormConvLayer(
        (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
        (activation): GELUActivation()
        (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
      )
      (1-4): 4 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
      (5-6): 2 x Wav2Vec2NoLayerNormConvLayer(
        (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        (activation): GELUActivation()
      )
    )
  )
  (feature_projection): Wav2Vec2FeatureProjection(
    (layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
    (projection): Linear(in_features=512, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Wav2Vec2Encoder(
    (pos_conv_embed): Wav2Vec2PositionalConvEmbedding(
  

This block builds a list of audio files that still need embeddings extracted.

It skips files that have already been processed and saved as .pt.

It also labels each audio file automatically:

Label (real/fake) is found from the immediate parent folder.

Split (training/validation/testing) is found from the grandparent folder.

The list audio_paths will later be fed into the Wav2Vec2 model for feature extraction.

In [4]:
# skip logic and audio file collection 

# list of processed files 
already_processed = set(os.listdir(SAVE_DIR))

# storing paths to audio files that need to be processed 
audio_paths = []

# all files inside folder 
for root, dirs, files in os.walk(DATA_DIR):
    for file in files:
        if file.endswith('.wav') or file.endswith('.mp3'):
            full_path = os.path.join(root, file)
            # infer label from folder name 
            label = os.path.basename(os.path.dirname(full_path))
            # infer data split 
            split = os.path.basename(os.path.dirname(os.path.dirname(full_path)))

            # convert to .pt
            save_name = file.replace(".wav", ".pt").replace(".mp3", ".pt")
            if save_name not in already_processed:
                audio_paths.append((full_path, label, split))

# number of files to be processed 
total_files = len(audio_paths)
print(f"Found {total_files} files to process.")


Found 69300 files to process.


## Feature Extraction 

For each audio file, I:

* Load it, resample to 16kHz if needed.

* Pass it through Wav2Vec2 to get a fixed-size vector (embedding).

* Save the vector along with its label (real/fake) and split (train/val/test).

* Batching is not done here — we do one file at a time because audio files can be different lengths.

In [5]:
# feature extraction loop 
# start a timer 

start_time = time.time()

# progress bar! 
for idx in tqdm(range(total_files), desc="Extracting features"):
    
    # current file info 
    path, label, split = audio_paths[idx]
    
    try:
        # load audio file 
        speech_array, sampling_rate = torchaudio.load(path)
        speech_array = speech_array.squeeze(0) 

        # resample if audio isnt 16 kHz (for wav2vec2)
        if sampling_rate != 16000:
            resampler = torchaudio.transforms.Resample(sampling_rate, 16000)
            speech_array = resampler(speech_array)

        # preprocess audio for wav2vec2 input 
        input_values = processor(speech_array.numpy(), sampling_rate=16000, return_tensors="pt").input_values.to(device)

        # forward pass through wav2vec2 without updating weights 
        with torch.no_grad():
            embeddings = model(input_values).last_hidden_state.mean(dim=1).squeeze(0) # avg over time dimension to get 1 fixed length vector 
        
        # save name 
        save_name = os.path.basename(path).replace(".wav", ".pt").replace(".mp3", ".pt")
        
        # save embedding along with label and split 
        torch.save({
            "embedding": embeddings.cpu(), # move to cpu before saving 
            "label": label,
            "split": split
        }, os.path.join(SAVE_DIR, save_name))
    
    except Exception as e:
        # catch and print errors to prevent crashing 
        print(f"Error processing {path}: {e}")

    # print progress every 100 files 
    if (idx + 1) % 100 == 0 or (idx + 1) == total_files:
        elapsed = time.time() - start_time
        print(f"Processed {idx + 1}/{total_files} - Elapsed {elapsed/60:.1f} minutes.")

# print once finished 
print("\nDone extracting and saving embeddings!")


Extracting features:   0%|          | 0/69300 [00:00<?, ?it/s]/stor/home/spl742/wavenv/lib/python3.9/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:861: UserWarning: 1Torch was not compiled with memory efficient attention. (Triggered internally at ../aten/src/ATen/native/transformers/hip/sdp_utils.cpp:505.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Extracting features:   0%|          | 101/69300 [00:09<1:29:06, 12.94it/s]

Processed 100/69300 - Elapsed 0.2 minutes.


Extracting features:   0%|          | 201/69300 [00:16<1:19:51, 14.42it/s]

Processed 200/69300 - Elapsed 0.3 minutes.


Extracting features:   0%|          | 301/69300 [00:25<1:24:52, 13.55it/s]

...

Processed 69100/69300 - Elapsed 237.6 minutes.


Extracting features: 100%|█████████▉| 69200/69300 [3:58:10<00:24,  4.03it/s]

Processed 69200/69300 - Elapsed 238.2 minutes.


Extracting features: 100%|██████████| 69300/69300 [3:58:40<00:00,  4.84it/s]

Processed 69300/69300 - Elapsed 238.7 minutes.

Done extracting and saving embeddings!
